In [ ]:
## •	50 teams tracked across 10 years (2000–2009)
## •	Includes WinRate, AvgAge, Budget, TeamRating, and Injuries

In [ ]:
# ⚙️ Install Required Packages
!pip install linearmodels statsmodels --quiet

# 📚 Import Libraries
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.formula.api import ols
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# 📁 Load Data
df = pd.read_csv("Sports_Team_Performance_Dataset__Balanced_.csv")
df.dropna(inplace=True)
df['TeamID'] = df['TeamID'].astype('category')
df['Year'] = pd.to_numeric(df['Year'])

# 🧱 Set Panel Index
df = df.set_index(['TeamID', 'Year'])

# 🎯 Define Regression Formula
dependent = 'TeamRating'
independent_vars = ['WinRate', 'AvgAge', 'Budget']
formula = dependent + ' ~ ' + ' + '.join(independent_vars)

# 🔁 Fixed Effects Model
fe_model = PanelOLS.from_formula(formula + ' + EntityEffects', data=df).fit()
print("\n📌 Fixed Effects Summary:\n", fe_model)

# 🔁 Random Effects Model
re_model = RandomEffects.from_formula(formula, data=df).fit()
print("\n📌 Random Effects Summary:\n", re_model)

# 🧪 Hausman Test
def hausman(fe, re):
    b = fe.params[independent_vars]
    B = re.params[independent_vars]
    v_b = fe.cov.loc[independent_vars, independent_vars]
    v_B = re.cov.loc[independent_vars, independent_vars]
    stat = (b - B).T @ np.linalg.inv(v_b - v_B) @ (b - B)
    pval = 1 - stats.chi2.cdf(stat, len(b))
    return stat, pval

hausman_stat, p_val = hausman(fe_model, re_model)
print(f"\n🔍 Hausman Test Statistic: {hausman_stat:.3f}, p-value: {p_val:.3f}")
print("➡️ Use Fixed Effects" if p_val < 0.05 else "➡️ Use Random Effects")


📌 Fixed Effects Summary:
                           PanelOLS Estimation Summary                           
Dep. Variable:             TeamRating   R-squared:                        0.0011
Estimator:                   PanelOLS   R-squared (Between):              0.0113
No. Observations:                 500   R-squared (Within):               0.0011
Date:                Thu, May 29 2025   R-squared (Overall):              0.0113
Time:                        20:32:47   Log-likelihood                   -1231.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.1594
Entities:                          50   P-value                           0.9236
Avg Obs:                      10.0000   Distribution:                   F(3,447)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             0.1594
 

/usr/local/lib/python3.11/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/data.py:680: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to

In [ ]:
print("\n--- FIXED EFFECTS MODEL ---")
print(fe_model.summary)

print("\n--- RANDOM EFFECTS MODEL ---")
print(re_model.summary)

hausman_stat, p_val = hausman(fe_model, re_model)
print(f"\nHausman test: stat={hausman_stat:.3f}, p={p_val:.3f}")
print("➡️ Use Fixed Effects" if p_val < 0.05 else "➡️ Use Random Effects")



--- FIXED EFFECTS MODEL ---
                          PanelOLS Estimation Summary                           
Dep. Variable:             TeamRating   R-squared:                        0.0011
Estimator:                   PanelOLS   R-squared (Between):              0.0113
No. Observations:                 500   R-squared (Within):               0.0011
Date:                Thu, May 29 2025   R-squared (Overall):              0.0113
Time:                        20:32:47   Log-likelihood                   -1231.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.1594
Entities:                          50   P-value                           0.9236
Avg Obs:                      10.0000   Distribution:                   F(3,447)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             0.1594